In [ ]:
# !pip install numpy scipy trimesh open3d plotly --quiet

In [ ]:
from pathlib import Path
import numpy as np
import trimesh
import open3d as o3d
from dataclasses import dataclass
from typing import Tuple
try:
    from scipy.interpolate import RBFInterpolator
except Exception:
    RBFInterpolator = None  # если нет SciPy, можно работать только с полиномом

# Вход/выход
IN_OBJ  = Path("input/broken_surface.obj")   # поменяй на свой
OUT_DIR = Path("output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def pca_frame(points: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Возвращает центр (3,) и ортонормальный базис (3x3): [u_axis, v_axis, n_axis]
    где u,v — главные оси в плоскости, n — нормаль (третья ось).
    """
    c = points.mean(axis=0)
    X = points - c
    # ковариация
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    # строки Vt — ортонорм. базис по убыванию дисперсии
    # возьмём две главные для плоскости и третью как нормаль
    e0, e1, e2 = Vt  # e0 — самая "растянутая"
    # гарантируем правую тройку
    if np.linalg.det(np.stack([e0, e1, e2], axis=1)) < 0:
        e2 = -e2
    B = np.stack([e0, e1, e2], axis=1)  # 3x3
    return c, B

def world_to_local(points: np.ndarray, c: np.ndarray, B: np.ndarray) -> np.ndarray:
    # локальные coords: [u, v, w] где w ~ высота над плоскостью
    return (points - c) @ B

def local_to_world(local_pts: np.ndarray, c: np.ndarray, B: np.ndarray) -> np.ndarray:
    return local_pts @ B.T + c

def make_grid_uv(R=10.0, N=200):
    u = np.linspace(-R, R, N)
    v = np.linspace(-R, R, N)
    U, V = np.meshgrid(u, v, indexing='xy')
    return U, V  # формы [N,N]

In [ ]:
def poly_design(u, v, deg=2):
    # базис мономов: 1, u, v, u^2, u v, v^2, ...
    terms = []
    for i in range(deg+1):
        for j in range(deg+1 - i):
            terms.append((i, j))
    Phi = np.stack([ (u**i)*(v**j) for (i,j) in terms ], axis=-1)
    return Phi, terms

def huber_weights(r, c=1.345):
    # Huber: |r|<=c -> 1 ; иначе c/|r|
    a = np.abs(r)
    w = np.ones_like(a)
    mask = a > c
    w[mask] = (c / (a[mask] + 1e-12))
    return w

def robust_poly_fit(u, v, w, deg=2, iters=5):
    """
    u,v,w — массивы наблюдений в локальной СК; подгоняем w ≈ f(u,v).
    Возвращает coeffs и функцию eval(u_grid,v_grid).
    """
    Phi, terms = poly_design(u, v, deg=deg)  # [N, M]
    y = w
    # начальный МНК
    coeffs, *_ = np.linalg.lstsq(Phi, y, rcond=None)

    for _ in range(iters):
        r = y - Phi @ coeffs
        # робастная шкала через MAD
        s = 1.4826 * np.median(np.abs(r - np.median(r))) + 1e-12
        ws = huber_weights(r / s)
        # Взвешенный МНК
        W = np.sqrt(ws)[:, None]
        Phi_w = Phi * W
        y_w   = y * W[:,0]
        coeffs, *_ = np.linalg.lstsq(Phi_w, y_w, rcond=None)

    def eval_on(U, V):
        Phi_g, _ = poly_design(U, V, deg=deg)  # broadcasting meshgrid
        # Phi_g shape [N,N,M] — превратим в [N*N,M]
        N1, N2, M = Phi_g.shape[0], Phi_g.shape[1], Phi_g.shape[2]
        Z = Phi_g.reshape(-1, M) @ coeffs
        return Z.reshape(N1, N2)
    return coeffs, eval_on

In [ ]:
def rbf_fit(u, v, w, kernel='thin_plate_spline', neighbors=100, epsilon=None):
    """
    Быстрая RBF через локальные соседи (scipy >= 1.10).
    kernel: 'thin_plate_spline' | 'multiquadric' | 'gaussian' | ...
    neighbors: локальная аппроксимация для масштабируемости.
    """
    if RBFInterpolator is None:
        raise RuntimeError("SciPy не установлен — RBF недоступен")
    X = np.stack([u, v], axis=1)
    rbf = RBFInterpolator(X, w, kernel=kernel, neighbors=neighbors, epsilon=epsilon)
    def eval_on(U, V):
        UV = np.stack([U.ravel(), V.ravel()], axis=1)
        Z = rbf(UV).reshape(U.shape)
        return Z
    return rbf, eval_on

In [ ]:
def grid_tris(N):
    """
    Индексация треугольников на регулярной сетке N×N (по U,V).
    """
    I = []
    for i in range(N-1):
        for j in range(N-1):
            a = i*N + j
            b = a + 1
            c = a + N
            d = c + 1
            I.append([a, c, b])
            I.append([b, c, d])
    return np.array(I, dtype=np.int32)

def save_patch_as_obj(verts_world, faces, out_path: Path):
    tm = trimesh.Trimesh(vertices=verts_world, faces=faces, process=False)
    tm.export(out_path)
    return out_path

In [ ]:
# --- заменяет предыдущую rbf_fit и reconstruct_patch_from_points (или добавляет рядом) ---
from dataclasses import dataclass
import numpy as np
from typing import Tuple
import trimesh

try:
    from scipy.interpolate import RBFInterpolator
except Exception:
    RBFInterpolator = None

@dataclass
class PatchConfig:
    R: float = 10.0
    N: int = 160
    method: str = "rbf-tps"   # 'poly2' | 'poly3' | 'rbf-tps' | 'rbf-mq'
    support_margin: float = 5.0
    rbf_neighbors: int = 200
    rbf_smoothing: float = 1e-3        # сглаживание для RBF
    max_support_points: int = 5000     # даунсэмпл опорной окрестности для устойчивости
    rbf_epsilon: float | None = None   # для 'rbf-mq' или 'gaussian'; для 'tps' игнорируется

def _pca_frame(points: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    c = points.mean(axis=0)
    X = points - c
    _, _, Vt = np.linalg.svd(X, full_matrices=False)
    e0, e1, e2 = Vt
    if np.linalg.det(np.stack([e0, e1, e2], axis=1)) < 0:
        e2 = -e2
    B = np.stack([e0, e1, e2], axis=1)
    return c, B

def _world_to_local(points, c, B): return (points - c) @ B
def _local_to_world(points, c, B): return points @ B.T + c

def _poly_design(u, v, deg=2):
    terms = [(i,j) for i in range(deg+1) for j in range(deg+1-i)]
    Phi = np.stack([(u**i)*(v**j) for (i,j) in terms], axis=-1)
    return Phi, terms

def _robust_poly_fit(u, v, w, deg=2, iters=5):
    Phi, _ = _poly_design(u,v,deg)
    y = w
    coeffs, *_ = np.linalg.lstsq(Phi, y, rcond=None)
    for _ in range(iters):
        r = y - Phi @ coeffs
        s = 1.4826*np.median(np.abs(r - np.median(r))) + 1e-12
        a = np.abs(r)/s
        ws = np.ones_like(a); mask = a>1.345; ws[mask] = 1.345/(a[mask]+1e-12)
        W = np.sqrt(ws)[:,None]
        coeffs, *_ = np.linalg.lstsq(Phi*W, (y*W[:,0]), rcond=None)
    def eval_on(U,V):
        Pg,_ = _poly_design(U,V,deg)
        N1,N2,M = Pg.shape
        Z = Pg.reshape(-1,M) @ coeffs
        return Z.reshape(N1,N2)
    return coeffs, eval_on

def _rbf_fit(u, v, w, kernel='thin_plate_spline', neighbors=200, smoothing=1e-3, epsilon=None):
    if RBFInterpolator is None:
        raise RuntimeError("SciPy не установлен — RBF недоступен")
    X = np.stack([u,v], axis=1)
    neighbors = int(min(neighbors, len(X)-1)) if len(X)>1 else 1
    rbf = RBFInterpolator(X, w, kernel=kernel, neighbors=neighbors,
                          smoothing=float(smoothing), epsilon=epsilon)
    def eval_on(U,V):
        UV = np.stack([U.ravel(), V.ravel()], axis=1)
        Z = rbf(UV).reshape(U.shape)
        return Z
    return rbf, eval_on

def _grid_uv(R=10.0, N=160):
    u = np.linspace(-R, R, N)
    v = np.linspace(-R, R, N)
    return np.meshgrid(u, v, indexing='xy')

def _grid_tris(N):
    tri = []
    for i in range(N-1):
        for j in range(N-1):
            a = i*N + j; b = a+1; c = a+N; d = c+1
            tri.append([a,c,b]); tri.append([b,c,d])
    return np.asarray(tri, np.int32)

def reconstruct_patch_from_points(points_world: np.ndarray, cfg: PatchConfig, out_path: Path) -> Path:
    # 1) локальная СК
    c,B = _pca_frame(points_world)
    P = _world_to_local(points_world, c, B)  # (u,v,w)

    # 2) поддержка: квадрат [-R-margin, R+margin]^2
    Rm = cfg.R + cfg.support_margin
    m = (np.abs(P[:,0])<=Rm) & (np.abs(P[:,1])<=Rm)
    Ps = P[m]
    if len(Ps) < 50:
        raise ValueError(f"Слишком мало опорных точек ({len(Ps)}). Увеличь support_margin или проверь масштаб.")

    # даунсэмпл для устойчивости
    if len(Ps) > cfg.max_support_points:
        idx = np.random.choice(len(Ps), cfg.max_support_points, replace=False)
        Ps = Ps[idx]

    u, v, w = Ps[:,0], Ps[:,1], Ps[:,2]
    print(f"[diagnostics] support pts: {len(Ps)}, uv-range: "
          f"u∈[{u.min():.3f},{u.max():.3f}], v∈[{v.min():.3f},{v.max():.3f}]")

    # 3) модель поверхности
    method = cfg.method.lower()
    if method == "poly2":
        _, f_eval = _robust_poly_fit(u,v,w,deg=2)
    elif method == "poly3":
        _, f_eval = _robust_poly_fit(u,v,w,deg=3)
    elif method == "rbf-tps":
        _, f_eval = _rbf_fit(u,v,w,kernel='thin_plate_spline',
                             neighbors=cfg.rbf_neighbors, smoothing=cfg.rbf_smoothing)
    elif method == "rbf-mq":
        _, f_eval = _rbf_fit(u,v,w,kernel='multiquadric',
                             neighbors=cfg.rbf_neighbors, smoothing=cfg.rbf_smoothing,
                             epsilon=cfg.rbf_epsilon)
    else:
        raise ValueError("method ∈ {'poly2','poly3','rbf-tps','rbf-mq'}")

    # 4) сетка [-R,R]
    U,V = _grid_uv(R=cfg.R, N=cfg.N)
    W = f_eval(U,V)

    # 5) вернуть в мир и сохранить
    UVW = np.stack([U,V,W], axis=-1).reshape(-1,3)
    verts_world = _local_to_world(UVW, c, B)
    faces = _grid_tris(cfg.N)
    trimesh.Trimesh(vertices=verts_world, faces=faces, process=False).export(out_path)
    return out_path

In [ ]:
# 1) загружаем исходные точки (возьмём вершины меша)
src = trimesh.load_mesh(str(IN_OBJ), process=False)
points_world = np.asarray(src.vertices)

# 2) конфиг: быстро и стабильно → 'poly2' или 'rbf-tps'
cfg = PatchConfig(
    R=1,
    N=160,             # можно 120-200
    method="rbf-tps",    # 'poly2' — очень быстрый; 'rbf-tps' — чуть плавнее
    support_margin=5.0,
    rbf_neighbors=200
)

OUT_PATCH = OUT_DIR / "patch_poly2.obj"
result_path = reconstruct_patch_from_points(points_world, cfg, OUT_PATCH)
print("✅ Patch saved to:", result_path)

[diagnostics] support pts: 5000, uv-range: u∈[-1.044,0.668], v∈[-0.585,0.378]
✅ Patch saved to: output/patch_poly2.obj


In [ ]:
import plotly.graph_objects as go

def show_mesh_plotly(obj_path):
    tm = trimesh.load_mesh(str(obj_path), process=False)
    x, y, z = tm.vertices.T
    i, j, k = tm.faces.T
    fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, color='lightgray', opacity=1.0)])
    fig.update_layout(scene=dict(aspectmode='data'), height=600)
    fig.show()

show_mesh_plotly(result_path)
show_mesh_plotly(IN_OBJ)